### Исследования в рамках третьего задания практикума ЭВМ
Выполнено Алескиным Александром

Подключим необходимые библиотеки.
Так же подключим реализованный класс MySVM. Переменные класса:

* X_ - матрица признаков обучаемой выборки
* y_ - ответы обцчаемой выборки
* C_ - параметр минимизации
* A_ - матрица коэффициентов двойственной задачи
* w_ - вектор весов
* b_ - смещение 
* ksi_ - коэффициенты(переменные) прямой задачи
* sup\_vec_ - опорные вектора
* gamma_ - параметр rbf ядра
* obj\_curve_ - значение целевой функции на последней итерации(или вектор при проверки в стохастическом варианте)



Интерфейс класса:

* fit(X, y, C) $-$ функция инициализации переменных класса. Х - матрица признаков, у - ответы, С - константа
* svm_qp_primal_solver(tol= 1e-6, max_iter= 100, verbose= False)  
* svm_qp_dual_solver(tol=1e-6, max_iter=100, verbose=False, gamma=0) 
* svm_liblinear_solver(tol= 1e-6, max_iter= 100, verbose= False)
* svm_libsvm_solver(tol = 1e-6, max_iter = 100,verbose = False, gamma = 0)
* compute_w()
* compute_support_vectors()
* predict(Y) $-$ классифицирует объекты по матрицы признаков Y
* visualize()
* compute_primal_objective()
* compute_dual_objective()
* svm_subgradient_solver(sw_start, w0 = 0, step = _const_, a = 0.01, b = 0.1, max_iter= 100, verbose= False, tol= 1e-6, rule = 'f', stochastic = False, func = 0, norma = 0, sub = 0.5) Аргументы функции: 
     * sw_start - начальное значение вектора весов
     * w0 - начальное значение смещения 
     * step - функция аргументов (номер итерации, параметр a, параметр b) для выбора длины шага (базовый вариант: $ a / it ^ b $) 
     * a и b - аргументы функции step 
     * rule - правило останова по изменению функционала('f') или по норме вектора весов('n')
     * stohastic - стохастический вариант ли
     * func и norma - значение для проверки правильности сходимости алгоритма(используется при verbose = True)
     * sub - доля объектов для стохастического метода(на основании которых делается шаг)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
direct = '/home/alex/prak/ex3/'
sys.path.append(direct)
import megasvm as msvm

#### 1. Исследовать зависимость времени работы реализованных методов для решения задачи линейного SVM от размерности признакового пространства и числа объектов в обучающей выборке. Исследовать скорость сходимости методов. Сравнить методы по полученным значениям целевой функции.

Задамим данные:

In [240]:
features = np.array([2, 5, 10, 50, 100, 500, 1000])
objects = np.array([100, 250, 500, 1000])       
answers = np.zeros((features.size, objects.size, 6, 2)) # 6 - methods, 0 - time, 1 - function
svm = msvm.MySVM()
                   
method = [svm.svm_qp_primal_solver, svm.svm_qp_dual_solver, svm.svm_subgradient_solver,
          svm.svm_subgradient_solver, svm.svm_liblinear_solver, svm.svm_libsvm_solver]

titles = ['qp primal', 'qp dual', 'subgradient', 'stohastic subgtadient', 'liblinear', 'libsvm']

Генерируем для каждого значения данные и вычисляем искомые значения(так как в данном пункте геометрические особенности классов не играет значения, то выберем наиболее удобные):

In [142]:
for obj in range(objects.size):
    for ft in range(features.size):
        X = np.vstack((np.random.multivariate_normal(np.zeros((features[ft])), np.eye(features[ft]), objects[obj]),
                       np.random.multivariate_normal(np.ones((features[ft])), np.eye(features[ft]), objects[obj])))
        y = np.vstack((np.ones((objects[obj], 1)), - np.ones((objects[obj], 1))))
        svm.fit(X, y, 0.1)
        for i in range(6):
            if i == 3:
                result = method[i](stochastic = True, sub = 0,4)
            else:
                result = method[i]()
            answers[ft, obj, i, 0] = result['time']
            answers[ft, obj, i, 1] = svm.compute_primal_objective()

Пример визуализации:

In [4]:
ft = 0
obj = 0
X = np.vstack((np.random.multivariate_normal(np.zeros((features[ft])), np.eye(features[ft]), objects[obj]),
               np.random.multivariate_normal(np.ones((features[ft])), np.eye(features[ft]), objects[obj])))
y = np.vstack((np.ones((objects[obj], 1)), - np.ones((objects[obj], 1))))
svm.fit(X, y, 0.5)
for i in range(6):
    if i == 3:
        method[i](stochastic = True)
    else:
        method[i]()
    plt.title(titles[i])   
    svm.visualize()
    plt.savefig('/home/alex/prak/ex3/' + titles[i] + '.pdf')
    

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Изобразим на графике время работы, как функцию от размерности пространства при 1000 элементах и как функции от количества элементов при 100 признаках:

In [191]:
obj = 2
for i in range(6):
    plt.plot(range(len(features)), answers[:, obj, i, 0], '-o')
plt.title('Time of training with 1000 objects')
plt.legend(titles, loc='upper left')

plt.xticks(range(len(features)), features)
plt.ylabel('time, s')
plt.xlabel('number of features')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
#plt.savefig(direct + 'linear time, obj1000' + '.pdf')
plt.show()


In [188]:
ft = 4
for i in range(6):
    plt.plot(range(len(objects)), answers[ft, :, i, 0], '-o')
plt.title('Time of training with 100 features')
plt.legend(titles, loc='upper left')

plt.xticks(range(len(objects)), objects)
plt.ylabel('time, s')
plt.xlabel('number of objects')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
#plt.savefig(direct + 'linear time, feat100' + '.pdf')
plt.show()


Аналогично с функционалом:

In [216]:
obj = 2
for i in range(6):
    plt.plot(range(len(features)), answers[:, obj, i, 1], '-o')
plt.title('Value of training with 1000 objects')
plt.legend(titles, loc='lower left')

plt.xticks(range(len(features)), features)
plt.ylabel('value')
plt.xlabel('number of features')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
plt.savefig(direct + 'linear func, obj1000' + '.pdf')
plt.show()


In [244]:
ft = 4
for i in range(6):
    plt.plot(range(len(objects)), answers[ft, :, i, 1], '-o')
plt.title('Value of function in primal problem  with 100 features')
plt.legend(titles, loc='upper left')

plt.xticks(range(len(objects)), objects)
plt.ylabel('value')
plt.xlabel('number of objects')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
#plt.savefig(direct + 'linear func, obj1000' + '.pdf')
plt.show()


### 2. Провести эти исследования для случая SVM с RBF ядром для тех методов, где возможен ядровой переход.

Начальные данные:

In [251]:
features = np.array([2, 50, 100, 500, 1000])
objects = np.array([100, 250, 500, 1000]) # долго очень считается 
answers2 = np.zeros((features.size, objects.size, 2, 2)) # 2 - methods, 0 - time, 1 - function
method = [svm.svm_qp_dual_solver, svm.svm_libsvm_solver]
titles = [ 'qp dual with rbf kernel', 'libsvm with rbf kernel']

Вычисление:

In [252]:
for obj in range(objects.size):
    for ft in range(features.size):
        X = np.vstack((np.random.multivariate_normal(np.zeros((features[ft])), np.eye(features[ft]), objects[obj]),
                       np.random.multivariate_normal(2 * np.ones((features[ft])), np.eye(features[ft]), objects[obj])))
        y = np.vstack((np.ones((objects[obj], 1)), - np.ones((objects[obj], 1))))
        svm.fit(X, y, 0.5)
        for i in range(2):
            result = method[i](gamma = 0.5)
            answers2[ft, obj, i, 0] = result['time']
            answers2[ft, obj, i, 1] = svm.compute_dual_objective()

/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Графики:

In [220]:
ft = 0
obj = 0
X = np.vstack((np.random.multivariate_normal(np.zeros((features[ft])), np.eye(features[ft]), objects[obj]),
               np.random.multivariate_normal(np.ones((features[ft])), np.eye(features[ft]), objects[obj])))
y = np.vstack((np.ones((objects[obj], 1)), - np.ones((objects[obj], 1))))
svm.fit(X, y, 0.5)
for i in range(2):
    method[i](gamma = 0.5)
    plt.title(titles[i])   
    svm.visualize()
    plt.savefig('/home/alex/prak/ex3/' + titles[i] + '.pdf')

plt.show()


/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [245]:
obj = 2
for i in range(2):
    plt.plot(range(len(features)), answers2[:, obj, i, 0], '-o')
for i in range(6):
    if i == 1 or i == 5:
        plt.plot(range(len(features)), answers[[0,2,3,4,5], obj, i, 0], '--o')
plt.title('Time of training with 1000 objects')
plt.legend(['qp dual with rbf kernel', 'libsvm with rbf kernel', 'qp dual linear', 'libsvm linear'],
           loc='upper left')

plt.xticks(range(len(features)), features)
plt.ylabel('times,')
plt.xlabel('number of features')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
#plt.savefig(direct + 'rbf time, obj1000' + '.pdf')
plt.show()
plt.close()

In [235]:
for i in range(2):
    plt.plot(range(len(objects)), answers2[2, :, i, 0], '-o')
for i in range(6):
    if i == 1 or i == 5:
        plt.plot(range(len(objects)), answers[4, :4, i, 0], '--o')
plt.title('Time in dual problem  with 100 features and rbf kernel')
plt.legend(['qp dual with rbf kernel', 'libsvm with rbf kernel', 'qp dual linear', 'libsvm linear'],
           loc='upper left')

plt.xticks(range(len(objects)), objects)
plt.ylabel('value')
plt.xlabel('number of objects')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
plt.savefig(direct + 'rbf time, feat100' + '.pdf')
plt.show()
plt.close()

In [254]:
obj = 2
for i in range(2):
    plt.plot(range(len(features)), answers2[:, obj, i, 1], '-o')
plt.title('Function value in dual problem with 1000 objects')
plt.legend(titles, loc='upper left')

plt.xticks(range(len(features)), features)
plt.ylabel('value')
plt.xlabel('number of features')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
plt.savefig(direct + 'rbffuncobj1000' + '.pdf')
plt.show()
plt.close()

In [238]:
ft = 4
for i in range(2):
    plt.plot(range(len(objects)), answers2[ft, :, i, 1], '-o')
plt.title('Function in dual problem  with 100 features and rbf kernel')
plt.legend(titles, loc='upper left')

plt.xticks(range(len(objects)), objects)
plt.ylabel('value')
plt.xlabel('number of objects')
plt.yscale('log')
ax = np.array(plt.axis())
ax[3] *= 10
plt.axis(ax)
plt.savefig(direct + 'rbf func, func100' + '.pdf')
plt.show()
plt.close()

#### 3. Реализовать процедуру поиска оптимального значения параметра C и ширины RBF ядра с помощью кросс-валидации (можно воспользоваться библиотекой scikit-learn). Исследовать зависимость ошибки на валидационной выборке от значений этих параметров. Рассмотреть случаи хорошо и трудно разделимых выборок.

Исходные данные:

In [3]:
from sklearn.cross_validation import KFold

svm = msvm.MySVM()
C = np.linspace(0.1, 1, num = 10)
rbf  = np.linspace(0.1, 2, num = 20)
mistake = np.zeros((C.size, rbf.size))
obj = 800
obj_h = obj // 2
ft = 2
folds = KFold(obj, 4, shuffle= True, random_state= 241)

Случай хорошо разделимой выборки:

In [257]:
mistake = np.zeros((C.size, rbf.size))
X_pp =  np.random.multivariate_normal(2 * np.ones((2)), 2 * np.eye(2), obj_h * 20)
X_p = X_pp[np.where(np.linalg.norm(X_pp, axis = 1) > 1)]
X_p = X_p[:obj_h]
X_m = np.random.multivariate_normal(np.zeros((2)), 0.5 * np.eye(2), obj_h)
X = np.vstack((X_p, X_m))
y = np.vstack((np.ones((obj_h, 1)), - np.ones((obj_h, 1))))

for i in range(C.size):
    for j in range(rbf.size):
        for train, test in folds:
            svm.fit(X[train], y[train], C[i])
            svm.svm_libsvm_solver(gamma = rbf[j])
            predict = svm.predict(X[test])
            mistake[i, j] += np.sum(predict != y[test])/y[test].size
mistake /= 4

/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Визуализируем порядок ошибки:

In [261]:
plt.matshow(mistake)
plt.yticks(range(len(C)), C)
plt.xticks(range(len(rbf)), rbf)
plt.colorbar()
plt.title('Mistake of prediction with rbf kernel(easy sample)')
plt.xlabel('parameter gamma')
plt.ylabel('parameter C')
plt.savefig(direct + 'easymistake.pdf')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [262]:
i = np.argmin(mistake)
k = i // rbf.size
l = i % rbf.size
print('оптимальные значения при C = %f, Gamma = %f. и равно %f' % (C[k], rbf[l], mistake[k,l]))
svm.fit(X[train], y[train], C[k])
svm.svm_libsvm_solver( gamma = rbf[l])
svm.visualize()
plt.title('Separating of easy sample')
#plt.savefig(direct + 'easy_mistake_map.pdf')
plt.show()

оптимальные значения при C = 0.700000, Gamma = 0.200000. и равно 0.057500


/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


Cлучай сложной разделимости:

In [7]:
mistake = np.zeros((C.size, rbf.size))
X_p1 =  np.random.multivariate_normal(np.ones((2)), 1.5 * np.eye(2), obj_h )
#X_p2 =  np.random.multivariate_normal(-np.ones((2)), 0.7 * np.eye(2), obj_h // 2)
X_m = np.random.multivariate_normal(np.zeros((2)), np.eye(2), obj_h)
X = np.vstack((X_p1, X_m))
y = np.vstack((np.ones((obj_h, 1)), - np.ones((obj_h, 1))))

for i in range(C.size):
    for j in range(rbf.size):
        for train, test in folds:
            svm.fit(X[train], y[train], C[i])
            svm.svm_libsvm_solver(gamma = rbf[j])
            predict = svm.predict(X[test])
            mistake[i, j] += np.sum(predict != y[test])/y[test].size
mistake /= 4

/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [8]:
plt.matshow(mistake)
plt.yticks(range(len(C)), C)
plt.xticks(range(len(rbf)), rbf)
plt.colorbar()
plt.title('Mistake of prediction with rbf kernel(hard sample)')
plt.savefig(direct + 'hard_mistake.pdf')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [9]:
i = np.argmin(mistake)
k = i // rbf.size
l = i % rbf.size
print('оптимальные значения при C = %f, Gamma = %f. и равно %f' % (C[k], rbf[l], mistake[k,l]))
svm.fit(X[train], y[train], C[k])
svm.svm_libsvm_solver( gamma = rbf[l])
svm.visualize()
plt.title('Separating hard simple')
plt.savefig(direct + 'hard_mistake_map.pdf')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:209: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


оптимальные значения при C = 0.100000, Gamma = 0.700000. и равно 0.298750


#### 4. Сравнить (по скорости сходимости и точности решения) несколько стратегий выбора шага $ α_t $ в методе субградиентого спуска: α, α/t , $α/t^β$ , где α, β — некоторые константы, t — номер итерации.

В качестве точного значения выберем следующие значения:

Вообще говоря, шаг должен зависеть от количества объектов. Проверим на 3 выборках (в качестве точного значения выберем следующие значения):

In [33]:
objects = np.array([100, 500, 1000]) 
a = np.array([0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10])
b = np.linspace(0, 2, num = 21)
times = np.zeros((a.size, b.size, objects.size))
func = times.copy()
its = func.copy()


In [39]:
for i in range(objects.size):
    X_p = np.random.multivariate_normal(2 * np.ones((2)), np.eye(2), objects[i])
    X_m = np.random.multivariate_normal(np.zeros((2)), np.eye(2), objects[i])
    X = np.vstack((X_p, X_m))
    y = np.vstack((np.ones((objects[i], 1)), - np.ones((objects[i], 1))))
    svm.fit(X, y, 0.1)
    result = svm.svm_qp_primal_solver(tol = 1e-10, max_iter = 1000)
    val = result['objective_curve']
    for k in range(a.size):
        for l in range(b.size):  
            result = svm.svm_subgradient_solver(a = a[k], b = b[l], max_iter= 10000, verbose = True, func = val)
            times[k, l, i] = result['time']
            func[k, l, i] = result['objective_curve'][-1]
            its[k, l, i] = result['iter']
            

/home/alex/prak/ex3/megasvm.py:167: RuntimeWarning: overflow encountered in multiply
  w_i = w_i - g * step(it, a, b)
/home/alex/prak/ex3/megasvm.py:167: RuntimeWarning: invalid value encountered in subtract
  w_i = w_i - g * step(it, a, b)


Оценим сходимость:

In [41]:
plt.matshow(func[:, :, 0])
plt.yticks(range(len(a)), a)
plt.xticks(range(len(b)), b)
plt.colorbar()
plt.title('Function in primal problem with 200 objects')
plt.xlabel('b')
plt.ylabel('a')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [53]:
plt.matshow(func[:, :, 1])
plt.yticks(range(len(a)), a)
plt.xticks(range(len(b)), b)
plt.colorbar()
plt.title('Function in primal problem with 1000 objects')
plt.xlabel('b')
plt.ylabel('a')
plt.savefig(direct + 'subgrad_functional.pdf')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [43]:
plt.matshow(func[:, :, 2])
plt.yticks(range(len(a)), a)
plt.xticks(range(len(b)), b)
plt.colorbar()
plt.title('Function in primal problem with 2000 objects')
plt.xlabel('b')
plt.ylabel('a')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


Оценим количество итераций:

In [54]:
plt.matshow(its[:, :, 1])
plt.yticks(range(len(a)), a)
plt.xticks(range(len(b)), b)
plt.colorbar()
plt.title('Iterations in primal problem with 1000 objects')
plt.xlabel('b')
plt.ylabel('a')
plt.savefig(direct + 'subgrad_iters.pdf')
plt.show()

/home/alex/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


#### 5. Исследовать, как размер подвыборки, по которой считается субградиент, в методе стохастического субградиентного спуска влияет на скорость сходимости метода и на точность решения.

In [265]:
nup = np.linspace(0.01, 1, num = 50)
func  = np.zeros((nup.shape))
iters = func.copy()
times = func.copy()
objects = 500
for i in range(100):
    X_p = np.random.multivariate_normal(2 * np.ones((2)), np.eye(2), objects)
    X_m = np.random.multivariate_normal(np.zeros((2)), np.eye(2), objects)
    X = np.vstack((X_p, X_m))
    y = np.vstack((np.ones((objects, 1)), - np.ones((objects, 1))))
    svm.fit(X, y, 0.1)
    result = svm.svm_qp_primal_solver(tol = 1e-10, max_iter = 1000)
    val = result['objective_curve']
    for l in range(nup.size):  
        result = svm.svm_subgradient_solver(a = 0.01, b = 0.01 , max_iter= 10000, stochastic= True,
                                            verbose = True, func = val, sub = nup[l])
        times[l] += result['time']
        func[l] = result['objective_curve'][-1]
        iters[l] += result['iter']

times = times // 100
iters = iters // 100

In [269]:
plt.plot(nup, iters, '-r')
plt.xlabel('Part of ruled object in step')
plt.ylabel('Amount of iterations')
plt.title('Speed of stochastic method')
plt.legend(['iterations'], loc='upper right')
plt.yscale('log')
plt.savefig(direct + 'stochastic.pdf')
plt.show()

In [275]:
iters[[25, 39, 49]]

array([ 258.,  178.,  139.])

In [276]:
iters[9]

1553.0